# Advanced Visualization Techniques

This notebook demonstrates advanced visualization methods for protein structures.

Topics:
1. 3D structure rendering
2. Distance and contact maps
3. Confidence heatmaps
4. Comparison visualizations
5. Interactive plots with Plotly

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent))

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('Environment ready!')

## 1. Generate Sample Data

In [ ]:
# Create synthetic protein structure (helix)
n_residues = 50
t = np.linspace(0, 4*np.pi, n_residues)

# Alpha helix geometry
radius = 2.3
pitch = 1.5

coords = np.zeros((n_residues, 3))
coords[:, 0] = radius * np.cos(t)
coords[:, 1] = radius * np.sin(t)
coords[:, 2] = pitch * t

# Add some noise
coords += np.random.randn(n_residues, 3) * 0.2

print(f'Generated structure with {n_residues} residues')

## 2. 3D Structure Visualization

In [ ]:
fig = plt.figure(figsize=(15, 5))

# View 1: Backbone trace
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(coords[:, 0], coords[:, 1], coords[:, 2], 'b-', linewidth=2, alpha=0.7)
ax1.scatter(coords[:, 0], coords[:, 1], coords[:, 2], c=range(n_residues), 
           cmap='viridis', s=50, alpha=0.8)
ax1.set_xlabel('X (Å)')
ax1.set_ylabel('Y (Å)')
ax1.set_zlabel('Z (Å)')
ax1.set_title('Backbone Trace')

# View 2: C-alpha spheres
ax2 = fig.add_subplot(132, projection='3d')
scatter = ax2.scatter(coords[:, 0], coords[:, 1], coords[:, 2], 
                     c=range(n_residues), cmap='plasma', s=200, alpha=0.6)
ax2.set_xlabel('X (Å)')
ax2.set_ylabel('Y (Å)')
ax2.set_zlabel('Z (Å)')
ax2.set_title('C-alpha Representation')
plt.colorbar(scatter, ax=ax2, label='Residue Index')

# View 3: Tube representation
ax3 = fig.add_subplot(133, projection='3d')
for i in range(len(coords)-1):
    ax3.plot(coords[i:i+2, 0], coords[i:i+2, 1], coords[i:i+2, 2],
            color=plt.cm.coolwarm(i/n_residues), linewidth=4, alpha=0.8)
ax3.set_xlabel('X (Å)')
ax3.set_ylabel('Y (Å)')
ax3.set_zlabel('Z (Å)')
ax3.set_title('Tube Representation')

plt.tight_layout()
plt.show()

## 3. Distance and Contact Maps

In [ ]:
# Calculate distance matrix
distances = np.sqrt(np.sum((coords[:, None, :] - coords[None, :, :]) ** 2, axis=2))

# Create contact map (threshold at 8 Å)
contacts = (distances < 8.0).astype(float)
np.fill_diagonal(contacts, 0)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Distance map
im1 = axes[0].imshow(distances, cmap='viridis', interpolation='nearest')
axes[0].set_title('Distance Map', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Residue Index')
axes[0].set_ylabel('Residue Index')
plt.colorbar(im1, ax=axes[0], label='Distance (Å)')

# Contact map
im2 = axes[1].imshow(contacts, cmap='RdYlBu_r', interpolation='nearest')
axes[1].set_title('Contact Map (<8Å)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Residue Index')
axes[1].set_ylabel('Residue Index')
plt.colorbar(im2, ax=axes[1], label='Contact')

# Distance difference (comparing with slightly perturbed structure)
coords_alt = coords + np.random.randn(*coords.shape) * 1.0
distances_alt = np.sqrt(np.sum((coords_alt[:, None, :] - coords_alt[None, :, :]) ** 2, axis=2))
dist_diff = np.abs(distances - distances_alt)

im3 = axes[2].imshow(dist_diff, cmap='hot', interpolation='nearest')
axes[2].set_title('Distance Difference', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Residue Index')
axes[2].set_ylabel('Residue Index')
plt.colorbar(im3, ax=axes[2], label='|Δ Distance| (Å)')

plt.tight_layout()
plt.show()

## 4. Confidence Heatmap

In [ ]:
# Generate synthetic confidence scores
confidence_per_residue = np.random.beta(8, 2, n_residues)  # High confidence
confidence_pairwise = np.outer(confidence_per_residue, confidence_per_residue)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Per-residue confidence
axes[0].bar(range(n_residues), confidence_per_residue, color='steelblue', alpha=0.7)
axes[0].axhline(y=0.7, color='r', linestyle='--', label='High Confidence Threshold')
axes[0].set_xlabel('Residue Index', fontsize=12)
axes[0].set_ylabel('Confidence Score', fontsize=12)
axes[0].set_title('Per-Residue Confidence', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Pairwise confidence
im = axes[1].imshow(confidence_pairwise, cmap='RdYlGn', vmin=0, vmax=1, interpolation='nearest')
axes[1].set_xlabel('Residue Index', fontsize=12)
axes[1].set_ylabel('Residue Index', fontsize=12)
axes[1].set_title('Pairwise Distance Confidence', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=axes[1], label='Confidence')

plt.tight_layout()
plt.show()

# Statistics
print(f'Mean confidence: {confidence_per_residue.mean():.3f}')
print(f'Min confidence: {confidence_per_residue.min():.3f}')
print(f'High confidence residues (>0.7): {(confidence_per_residue > 0.7).sum()}/{n_residues}')

## 5. Interactive Plotly Visualization

In [ ]:
try:
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    # Create 3D scatter plot
    fig = go.Figure(data=[
        go.Scatter3d(
            x=coords[:, 0],
            y=coords[:, 1],
            z=coords[:, 2],
            mode='markers+lines',
            marker=dict(
                size=8,
                color=list(range(n_residues)),  # Convert range to list
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title='Residue')
            ),
            line=dict(color='darkblue', width=2),
            text=[f'Residue {i}' for i in range(n_residues)],
            hovertemplate='<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<br>Z: %{z:.2f}'
        )
    ])
    
    fig.update_layout(
        title='Interactive 3D Protein Structure',
        scene=dict(
            xaxis_title='X (Å)',
            yaxis_title='Y (Å)',
            zaxis_title='Z (Å)',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        width=900,
        height=700
    )
    
    fig.show()
    
except ImportError:
    print('Plotly not installed. Install with: pip install plotly')
    print('Skipping interactive visualization.')

## 6. Ramachandran Plot

In [ ]:
# Calculate backbone angles (simplified)
phi_angles = []
psi_angles = []

for i in range(1, n_residues-1):
    # Simplified dihedral calculation
    v1 = coords[i] - coords[i-1]
    v2 = coords[i+1] - coords[i]
    
    # Calculate angles
    phi = np.arctan2(v1[1], v1[0]) * 180 / np.pi
    psi = np.arctan2(v2[1], v2[0]) * 180 / np.pi
    
    phi_angles.append(phi)
    psi_angles.append(psi)

# Plot Ramachandran
plt.figure(figsize=(10, 8))

# Create density background
plt.hexbin(phi_angles, psi_angles, gridsize=20, cmap='Blues', alpha=0.3)

# Plot points
plt.scatter(phi_angles, psi_angles, c=range(len(phi_angles)), 
           cmap='viridis', s=100, edgecolors='black', linewidth=0.5, zorder=10)

# Ramachandran regions (approximate)
from matplotlib.patches import Rectangle
# Alpha-helix region
plt.gca().add_patch(Rectangle((-180, -70), 120, 70, fill=False, edgecolor='red', 
                              linewidth=2, linestyle='--', label='α-helix region'))
# Beta-sheet region
plt.gca().add_patch(Rectangle((-180, 90), 120, 90, fill=False, edgecolor='blue', 
                              linewidth=2, linestyle='--', label='β-sheet region'))

plt.xlabel('Phi (φ) angle (degrees)', fontsize=12)
plt.ylabel('Psi (ψ) angle (degrees)', fontsize=12)
plt.title('Ramachandran Plot', fontsize=14, fontweight='bold')
plt.xlim(-180, 180)
plt.ylim(-180, 180)
plt.grid(alpha=0.3)
plt.legend()
plt.colorbar(label='Residue Index')
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- Multiple 3D visualization techniques
- Distance and contact map generation
- Confidence score visualization
- Interactive plotting with Plotly
- Ramachandran analysis

These visualization tools are essential for:
- Quality assessment of predictions
- Identifying structural motifs
- Comparing predictions with ground truth
- Publication-ready figures